## セブンイレブンの店舗情報取得

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import requests, bs4, os, re, math, csv, codecs, tqdm
from time import sleep
from random import randint

base_urls = "http://www.cvsmap.com/shop/%E3%82%BB%E3%83%96%E3%83%B3%E3%82%A4%E3%83%AC%E3%83%96%E3%83%B3/{}.html"

#ページ数ループの計算
pages = math.ceil(9999/15)

In [2]:
#ページURLの作成
url = []
for e in range(pages):
    url.append(str(base_urls.format(e+1)))

In [ ]:
no = 1

for s in tqdm.tqdm(url):
    
    print('Now on No.{}'.format(no))
    
    res = requests.get(s)
    res.raise_for_status()
    
    soup = bs4.BeautifulSoup(res.text, "lxml")
    
    shop_div = soup.findAll('div', class_="shop_box")
    
    for i in shop_div:

        shop_name = i.find('p').text
        shop_loc = i.find('p', class_="shop_address").text
        sdata = {'shop_name':shop_name,'shop_loc':shop_loc}
        
        
        # ファイルオープン
        f = codecs.open('output_shop.csv', 'a', 'utf-8')
        writer = csv.writer(f, lineterminator='\n')
        # データをリストに保持
        csvlist = []
        csvlist.append(shop_name)
        csvlist.append(shop_loc)   
        # 出力
        writer.writerow(csvlist)
        # ファイルクローズ
        f.close()
        
    no += 1
    sleep(randint(1,5))

## データフレーム化＆緯度経度取得

In [37]:
import pandas as pd
from pandas import DataFrame
import json, requests, numpy

df = pd.read_csv("output_shop.csv", names=('店舗名', '住所'))

In [38]:
df.shape

(585, 2)

In [39]:
df.head()

,店舗名,住所
0,セブンイレブン 名古屋椿町店,愛知県名古屋市中村区椿町１４－２３
1,セブンイレブン品川駅前店新川屋,東京都港区高輪３丁目２５－２０
2,セブンイレブン 京都駅八条口店,京都府京都市下京区東塩小路釜殿町３１－１
3,セブンイレブン浦安舞浜店,千葉県浦安市舞浜２丁目４６－１
4,セブンイレブングランフロント大阪店,大阪府大阪市北区大深町３－１


In [43]:
df['緯度'] = numpy.NAN
df['経度'] = numpy.NAN

In [ ]:
for loc, num in tqdm.tqdm(zip(df['住所'], range(len(df['住所'])))):
    address = loc
    key = 'YOURKEY'
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}'.format(address, key)

    map_res = requests.get(url)
    seven_loc = json.loads(map_res.text)

    seven_lat = seven_loc['results'][0]['geometry']['location']['lat']
    seven_lng = seven_loc['results'][0]['geometry']['location']['lng']
    
    df['緯度'][num] = seven_lat
    df['経度'][num] = seven_lng

In [55]:
df.head()

,店舗名,住所,緯度,経度
0,セブンイレブン 名古屋椿町店,愛知県名古屋市中村区椿町１４－２３,35.169044,136.880325
1,セブンイレブン品川駅前店新川屋,東京都港区高輪３丁目２５－２０,35.630302,139.737793
2,セブンイレブン 京都駅八条口店,京都府京都市下京区東塩小路釜殿町３１－１,34.984692,135.757333
3,セブンイレブン浦安舞浜店,千葉県浦安市舞浜２丁目４６－１,35.639246,139.882497
4,セブンイレブングランフロント大阪店,大阪府大阪市北区大深町３－１,34.705345,135.494643


In [56]:
# CSV出力
df.to_csv("shop_loc.csv", sep=",")

## 地図上にプロット

In [65]:
%matplotlib inline
import folium

seven_map = folium.Map(location=[df['緯度'].mean(), df['経度'].mean()], zoom_start=5)

In [66]:
coordinates = []

loc = list(df['緯度'])
lng = list(df['経度'])

for i, k in zip(loc, lng):
    coordinates.append((i, k))

In [67]:
#マーカー追加
for i in range(0,len(coordinates)):

    folium.Marker(coordinates[i],
                 ).add_to(seven_map)

seven_map